In [0]:
# tfidf intro

In [105]:
# imports 
import os
import re 
import numpy as np
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [79]:
#nlp tools
import spacy
print('spaCy Version: %s' % (spacy.__version__))
spacy_nlp = spacy.load('en_core_web_sm')

spaCy Version: 2.1.9


In [90]:
!pip install num2words

     |████████████████████████████████| 102kB 2.3MB/s 


In [0]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from collections import Counter
from num2words import num2words

In [0]:
import nltk
import os
import string
import numpy as np
import copy
import pandas as pd
import pickle
import re
import math

In [0]:
title = "stories"
alpha = 0.3

### intro to tfidf and formulea

## Data

In [69]:
!ls

sample_data  stories  stories.zip  stories.zip.1


In [0]:
! rm stories.zip.1

In [0]:
!wget http://archives.textfiles.com/stories.zip

In [0]:
!unzip stories.zip

In [72]:
!ls

sample_data  stories  stories.zip


In [6]:
!pwd

/content


In [0]:
folders = [x[0] for x in os.walk(str(os.getcwd())+'/stories/')]

In [74]:
folders

['/content/stories/', '/content/stories/SRE', '/content/stories/FARNON']

In [0]:
folders = folders[:len(folders)-1]

In [76]:
folders

['/content/stories/', '/content/stories/SRE']

In [0]:
dataset = []
c = False
for i in folders:
  file = open(i+"/index.html",'r')
  text = file.read().strip()
  file.close()

  file_name = re.findall('><A HREF="(.*)">',text)
  file_title = re.findall('<BR><TD> (.*)\n',text)

  if c == False:
    file_name = file_name[2:]
    c = True

  for j in range(len(file_name)):
    dataset.append((str(i) + '/'+str(file_name[j]), file_title[j]))

In [109]:
N = len (dataset)
N

467

In [0]:
def print_doc(id):
    print(dataset[id])
    file = open(dataset[id][0], 'r', encoding='cp1250')
    text = file.read().strip()
    file.close()
    print(text)

## Preprocessing

In [0]:
def convert_lower_case(text):
  new_text = np.char.lower(text)
  return new_text


In [0]:
def remove_stop_words(text):
  doc = spacy_nlp(str(text))
  tokens = [token.text for token in doc if not token.is_stop]
  new_data = ' '.join(tokens)
  return new_data


In [0]:
def remove_punctuation(data):
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in range(len(symbols)):
        data = np.char.replace(data, symbols[i], ' ')
        data = np.char.replace(data, "  ", " ")
    data = np.char.replace(data, ',', '')
    return data



In [0]:
def remove_apostrophe(data):
    return np.char.replace(data, "'", "")

In [0]:
def stemming(data):
    stemmer= PorterStemmer()
    
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        new_text = new_text + " " + stemmer.stem(w)
    return new_text

In [0]:
def convert_numbers(data):
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        try:
            w = num2words(int(w))
        except:
            a = 0
        new_text = new_text + " " + w
    new_text = np.char.replace(new_text, "-", " ")
    return new_text

In [0]:
def preprocess(data):
    data = convert_lower_case(data)
    data = remove_punctuation(data) #remove comma seperately
    data = remove_apostrophe(data)
    data = remove_stop_words(data)
    data = convert_numbers(data)
    data = stemming(data)
    data = remove_punctuation(data)
    data = convert_numbers(data)
    data = stemming(data) #needed again as we need to stem the words
    data = remove_punctuation(data) #needed again as num2word is giving few hypens and commas fourty-one
    data = remove_stop_words(data) #needed again as num2word is giving stop words 101 - one hundred and one
    return data

In [0]:
## Extracting Data

In [0]:
%%time
processed_text = []
processed_title = []

for i in dataset[:N]:
    file = open(i[0], 'r', encoding="utf8", errors='ignore')
    text = file.read().strip()
    file.close()

    processed_text.append(word_tokenize(str(preprocess(text))))
    processed_title.append(word_tokenize(str(preprocess(i[1]))))

In [149]:
dataset[1]

('/content/stories//13chil.txt', 'The Story of the Sly Fox')

In [119]:
len(processed_text)

467


## Calculating DF for all words

In [0]:
DF = {}

for i in range(N):
    tokens = processed_text[i]
    for w in tokens:
        try:
            DF[w].add(i)
        except:
            DF[w] = {i}

    tokens = processed_title[i]
    for w in tokens:
        try:
            DF[w].add(i)
        except:
            DF[w] = {i}
for i in DF:
    DF[i] = len(DF[i])

In [122]:
len(DF)

32172

In [123]:
total_vocab_size = len(DF)
total_vocab_size

32172

In [0]:
total_vocab = [x for x in DF]

In [128]:
print(total_vocab[0:20])

['sharewar', 'trial', 'project', 'freewar', 'need', 'support', 'continu', 'hundr', 'west', 'fifti', 'north', 'jim', 'prentic', 'copyright', 'thousand', 'nineti', 'brandon', 'manitoba', 'canada', 'magic']


In [0]:
def doc_freq(word):
    c = 0
    try:
        c = DF[word]
    except:
        pass
    return c

## Calculating TF-IDF for body, we will consider this as the actual tf-idf as we will add the title weight to this.

In [0]:
doc = 0

tf_idf = {}

for i in range(N):
    
    tokens = processed_text[i]
    
    counter = Counter(tokens + processed_title[i])
    words_count = len(tokens + processed_title[i])
    
    for token in np.unique(tokens):
        
        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = np.log((N+1)/(df+1))
        
        tf_idf[doc, token] = tf*idf

    doc += 1

## Calculating TF-IDF for Title

In [0]:
tf_idf_title = {}

for i in range(N):
    
    tokens = processed_title[i]
    counter = Counter(tokens + processed_text[i])
    words_count = len(tokens + processed_text[i])

    for token in np.unique(tokens):
        
        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = np.log((N+1)/(df+1))
        
        tf_idf_title[doc, token] = tf*idf

    doc += 1

In [0]:
tf_idf[(1, 'bag')]

In [0]:
tf_idf_title[(1, 'bag')]

##  Merging the TF-IDF according to weights

In [0]:
for i in tf_idf:
    tf_idf[i] *= alpha

In [0]:
for i in tf_idf_title:
    tf_idf[i] = tf_idf_title[i]

In [143]:
len(tf_idf)

320256

# TF-IDF Cosine Similarity Ranking

In [0]:
def cosine_sim(a, b):
    cos_sim = np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
    return cos_sim

### Vectorising tf-idf

In [0]:
D = np.zeros((N, total_vocab_size))
for i in tf_idf:
    try:
        ind = total_vocab.index(i[1])
        D[i[0]][ind] = tf_idf[i]
    except:
        pass

In [0]:
def gen_vector(tokens):

    Q = np.zeros((len(total_vocab)))
    
    counter = Counter(tokens)
    words_count = len(tokens)

    query_weights = {}
    
    for token in np.unique(tokens):
        
        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = math.log((N+1)/(df+1))

        try:
            ind = total_vocab.index(token)
            Q[ind] = tf*idf
        except:
            pass
    return Q

In [148]:

def cosine_similarity(k, query):
    print("Cosine Similarity")
    preprocessed_query = preprocess(query)
    tokens = word_tokenize(str(preprocessed_query))
    
    print("\nQuery:", query)
    print("")
    print(tokens)
    
    d_cosines = []
    
    query_vector = gen_vector(tokens)
    
    for d in D:
        d_cosines.append(cosine_sim(query_vector, d))
        
    out = np.array(d_cosines).argsort()[-k:][::-1]
    
    print("")
    
    print(out)

#     for i in out:
#         print(i, dataset[i][0])

Q = cosine_similarity(10, "Without the drive of Rebeccah's insistence, Kate lost her momentum. She stood next a slatted oak bench, canisters still clutched, surveying")

Cosine Similarity

Query: Without the drive of Rebeccah's insistence, Kate lost her momentum. She stood next a slatted oak bench, canisters still clutched, surveying

['drive', 'rebeccah', 'insist', 'kate', 'lost', 'momentum', 'stood', 'slat', 'oak', 'bench', 'canist', 'clutch', 'survey']

[200 433 166 175 169 211 402  63 290 151]
